In [ ]:
print("="*100)
print("공모전/대외활동 크롤링")
print("="*100)

In [1]:
#1. 정보 Init
import datetime
now = datetime.datetime.now()
query_txt = "데이터"
f_dir = "c://tmp//"

fx_name = f_dir + f'{query_txt}-{now.year}-{now.month:02}-{now.day:02}' + '.txt'

In [2]:
#2. driver 실행
from selenium import webdriver
import time

chrome_path = "c://tmp//chromedriver.exe"

options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(chrome_path,options=options)
driver.get("https://thinkyou.co.kr/index.asp")
driver.maximize_window() 
time.sleep(3)

In [3]:
#3. '공모전/대외활동' 마우스 위에 올리고, 클릭
from selenium.webdriver.common.action_chains import ActionChains

a = ActionChains(driver)
m= driver.find_element_by_xpath('//*[@id="gnb"]/li[1]/a/span')
a.move_to_element(m).perform()
time.sleep(0.5)

driver.find_element_by_xpath('//*[@id="gnb"]/li[1]/ul/li[1]').click()

In [4]:
#4. '검색'에서 공모전 클릭, '데이터' 입력
driver.find_element_by_xpath('//*[@id="searchFrm"]/div/div/div[1]/select').click()
driver.find_element_by_xpath('//*[@id="searchFrm"]/div/div/div[1]/select/option[2]').click()

driver.find_element_by_xpath('//*[@id="searchFrm"]/div/div/div[1]/span/input[1]').click()
elem = driver.find_element_by_name("searchstr")
elem.send_keys(query_txt)
elem.send_keys("\n")
time.sleep(2)

#일단 여러개의 데이터 긁어 오는 경우를 생각하여 '접수중'클릭 나중에 구현
#driver.find_element_by_xpath('//*[@id="contents"]/div[2]/div[1]/a[3]').click()
#time.sleep(1)

In [5]:
#5. url수집
from bs4 import BeautifulSoup
urls =[]
default_dir = "https://thinkyou.co.kr"
count = 0

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

all = soup.find('div','contestArea').find_all('div','title')

for i in all:
    a_tag = i.find('a')['href']
    urls.append(default_dir + a_tag)
    count+=1

In [6]:
#6. url접속 및 이미지 수집
import urllib.request
import urllib
import os
import shutil
import sys
from urllib.parse import quote
from selenium.webdriver.common.keys import Keys

def createDirectory(directory): 
    if os.path.exists(directory): #존재하면 제거후 생성
        shutil.rmtree(directory)
    os.mkdir(directory)

bmp_map = dict.fromkeys(range(0x10000,sys.maxunicode + 1),0xfffd)
count = 0
file_no = 1
image_path = f_dir + 'images//'
createDirectory(image_path)

for num in range(0,len(urls)):
    driver.get(urls[num]) #사이트 접속

    html1 = driver.page_source
    soup1 = BeautifulSoup(html1,'html.parser')
    
    #Title 수집
    title = soup1.find('dl','title').find('h1').get_text()
    
    f = open(fx_name,'a',encoding='UTF-8')
    f.write(title + '\n')
    f.close()
    
    # 이미지 확대
    driver.find_element_by_xpath('//*[@id="printArea"]/div[1]/div/div[1]/img').click()
    
    html2 = driver.page_source
    soup2 = BeautifulSoup(html2,'html.parser')
    
    
    #이미지 수집
    try:
        photo = soup2.find('div','galleryImg').find('img')['src']
    except:
        continue
    else:
        urllib.request.urlretrieve('https://thinkyou.co.kr/' + quote(photo),image_path + str(file_no) + '.jpg')
        file_no += 1
    
    time.sleep(1)
    
driver.close()

KeyboardInterrupt: 